In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

from peft import PeftModel
from huggingface_hub import login
import torch
from mikkel_secrets import secrets

login(secrets["llama"]["token"])

/home/mikkel/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/mikkel/.cache/huggingface/token
Login successful


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model_id = "meta-llama/Llama-2-7b-chat-hf"

compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    #torch_dtype=torch.bfloat16, 
    quantization_config=quant_config,
    device_map={"": 0},
)

model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.09s/it]


In [5]:
ft_model = PeftModel.from_pretrained(model, "results/checkpoint-105")


text = """

Create a LinkedIn post using the language and tone of Mikkel Jensen. 
The post is about how to asses impact of an ML or AI solution, before starting development and has the title: Assessing impact of ML & AI solutions.
"""

inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = ft_model.generate(**inputs, max_new_tokens=1000, max_time=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



Create a LinkedIn post using the language and tone of Mikkel Jensen. 
The post is about how to asses impact of an ML or AI solution, before starting development and has the title: Assessing impact of ML & AI solutions.

I’ve noticed a lot of people talking about how to develop ML & AI solutions, but not about how to assess their impact after deployment.

I think that’s a crucial part of the development process, as not every solution will be the best one.

Here are some ways to assess the impact of a ML & AI solution:

1. Performance metrics:
Measure the performance of the solution against its intended use case.

2. Comparison to baseline:
Compare the performance of the solution to a known baseline, e.g. a rule based system.

3. Causal impact analysis:
Analyze the impact of the solution by comparing the causal impact on the target variable, e.g. revenue, with and without the solution.

4. Post-deployment monitoring:
Continue monitoring the solution after deployment to ensure it’s perf